In [1]:
import torch
from IPython.display import clear_output
from transformers import BertTokenizer




# 記得我們是使用中文 BERT
model_version = 'bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(model_version)
magic_threshold = 0.6

# 情境句子
text_a = "[CLS]等到潮水[MASK]了,就知道誰沒穿褲子"
text_b = "等到潮水[MASK]了"
text_c = "就知道誰沒穿褲子"
text_d = "印度面臨「海嘯式」的新冠疫情，確診及死亡人數節節上升。剛離開印度回國的中國人蒙姐近日就其所見所聞娓娓道來，她認為，專家預測印度實際感染人數要比公佈的數字多3至5倍應是真的[MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK]在印度即使確診了也根本沒有溯源一說，沒有任何強制性核酸檢測，而據她觀察當地很多人對生死看得特別淡，心態與中國人大不同。"
text_f = "印度面臨「海嘯式」的新冠疫情，確診及死亡人數節節上升。剛離開印度回國的中國人蒙姐近日就其所見所聞娓娓道來，她認為，專家預測印度實際感染人數要比公佈的數字多3至5倍應是真的。在印度即使確診了也根本沒有溯源一說，沒有任何強制性核酸檢測，而據她觀察當地很多人對生死看得特別淡，心態與中國人大不同。"
#
def get_input_from_mask_sentence(a):
    tokens = tokenizer.tokenize(a)
    ids = tokenizer.convert_tokens_to_ids(tokens)
    
    # 除了 tokens 以外我們還需要辨別句子的 segment ids
    input_ids = torch.tensor([ids])  # (1, seq_len)
    token_type_ids = torch.zeros_like(input_ids)  # (1, seq_len)
    return tokens, input_ids ,token_type_ids

#
def get_input_from_two_sentence(a, b):
    inputs = tokenizer.encode_plus(a, b, return_tensors='pt', add_special_tokens=True)
    input_ids ,token_type_ids = inputs['input_ids'],inputs['token_type_ids']
    
    return input_ids ,token_type_ids

In [2]:
# 潤句1:一次偵測
# 適合用於微調語法
def convert2text(predictions,k=3):
    probs, indices = torch.topk(torch.softmax(predictions, -1), k)
    predicted_tokens = tokenizer.convert_ids_to_tokens(indices.tolist())
    return predicted_tokens,probs


from transformers import BertForMaskedLM

#
maskedLM_model = BertForMaskedLM.from_pretrained(model_version)
clear_output()

tokens, input_ids ,token_type_ids = get_input_from_mask_sentence(text_f)

# 使用 masked LM 估計 [MASK] 位置所代表的實際 token 
maskedLM_model.eval()
with torch.no_grad():
    outputs = maskedLM_model(input_ids, token_type_ids=token_type_ids)
    predictions = outputs[0]
    # (1, seq_len, num_hidden_units)
del maskedLM_model

k=3
for idx in range(0,len(tokens)):
    predicted_tokens,probs = convert2text(predictions[0,idx],k=k)
    boolresult= False
    for tmp_p,tmp_t in zip(probs,predicted_tokens):
        #
        if tokens[idx] == tmp_t:
            continue
        #
        if float(tmp_p) > magic_threshold or tokens[idx]=="[MASK]":
            print(tokens[idx],tmp_t,f" v ->{tmp_p}")
            boolresult = True
            break
    if boolresult is False:
        print(tokens[idx],tokens[idx])

印 印
度 度
面 面
臨 臨
「 「
海 海
嘯 嘯
式 式
」 」
的 的
新 新
冠 冠
疫 疫
情 情
， ，
確 確
診 診
及 及
死 死
亡 亡
人 人
數 數
節 節
節 節
上 上
升 升
。 。
剛 剛
離 離
開 開
印 印
度 度
回 回
國 來  v ->0.9020340442657471
的 的
中 中
國 國
人 人
蒙 蒙
姐 姐
近 近
日 日
就 將  v ->0.602899968624115
其 其
所 所
見 見
所 所
聞 聞
娓 娓
娓 娓
道 道
來 來
， ，
她 她
認 認
為 為
， ，
專 專
家 家
預 預
測 測
印 印
度 度
實 實
際 際
感 感
染 染
人 人
數 數
要 要
比 比
公 公
佈 佈
的 的
數 數
字 字
多 多
3 3
至 至
5 5
倍 倍
應 應
是 是
真 真
的 的
。 。
在 在
印 印
度 度
即 即
使 使
確 確
診 診
了 了
也 也
根 根
本 本
沒 沒
有 有
溯 溯
源 源
一 一
說 說
， ，
沒 沒
有 有
任 任
何 何
強 強
制 制
性 性
核 核
酸 酸
檢 檢
測 測
， ，
而 而
據 據
她 她
觀 觀
察 察
當 當
地 地
很 很
多 多
人 人
對 對
生 生
死 死
看 看
得 得
特 特
別 別
淡 淡
， ，
心 心
態 態
與 與
中 中
國 國
人 人
大 大
不 不
同 同
。 。


In [5]:
# 潤句2：逐字偵測
# 適合用於產生銜接句
def convert2text(predictions,k=1):
    probs, indices = torch.topk(torch.softmax(predictions, -1), k)
    predicted_tokens = tokenizer.convert_ids_to_tokens(indices.tolist())
    return predicted_tokens,probs


from transformers import BertForMaskedLM

#
maskedLM_model = BertForMaskedLM.from_pretrained(model_version)
clear_output()

tokens, input_ids ,token_type_ids = get_input_from_mask_sentence(text_d)
for idx,tmp_token in enumerate(tokens):

    if tokens[idx]=="[MASK]":
        tmp_text = tokens[:]
        tmp_text[idx]="[MASK]"
        tmp_tokens, tmp_input_ids ,tmp_token_type_ids = get_input_from_mask_sentence("".join(tmp_text))
        with torch.no_grad():
            outputs = maskedLM_model(tmp_input_ids, token_type_ids=tmp_token_type_ids)
            predictions = outputs[0]
            # (1, seq_len, num_hidden_units)
            pred_token,prob = convert2text(predictions[0,idx])
        print(f"{tokens[idx]},{pred_token[0]} --> {prob}")
        tokens[idx] = str(pred_token[0])
    else:
        print(f"{tokens[idx]},{tokens[idx]}")
del maskedLM_model

印,印
度,度
面,面
臨,臨
「,「
海,海
嘯,嘯
式,式
」,」
的,的
新,新
冠,冠
疫,疫
情,情
，,，
確,確
診,診
及,及
死,死
亡,亡
人,人
數,數
節,節
節,節
上,上
升,升
。,。
剛,剛
離,離
開,開
印,印
度,度
回,回
國,國
的,的
中,中
國,國
人,人
蒙,蒙
姐,姐
近,近
日,日
就,就
其,其
所,所
見,見
所,所
聞,聞
娓,娓
娓,娓
道,道
來,來
，,，
她,她
認,認
為,為
，,，
專,專
家,家
預,預
測,測
印,印
度,度
實,實
際,際
感,感
染,染
人,人
數,數
要,要
比,比
公,公
佈,佈
的,的
數,數
字,字
多,多
3,3
至,至
5,5
倍,倍
應,應
是,是
真,真
的,的
[MASK],。 --> tensor([0.4508])
[MASK],他 --> tensor([0.1258])
[MASK],說 --> tensor([0.1806])
[MASK],， --> tensor([0.3064])
[MASK],我 --> tensor([0.0894])
[MASK],們 --> tensor([0.3087])
[MASK],所 --> tensor([0.1716])
[MASK],見 --> tensor([0.2135])
[MASK],所 --> tensor([0.6250])
[MASK],聞 --> tensor([0.8029])
在,在
印,印
度,度
即,即
使,使
確,確
診,診
了,了
也,也
根,根
本,本
沒,沒
有,有
溯,溯
源,源
一,一
說,說
，,，
沒,沒
有,有
任,任
何,何
強,強
制,制
性,性
核,核
酸,酸
檢,檢
測,測
，,，
而,而
據,據
她,她
觀,觀
察,察
當,當
地,地
很,很
多,多
人,人
對,對
生,生
死,死
看,看
得,得
特,特
別,別
淡,淡
，,，
心,心
態,態
與,與
中,中
國,國
人,人
大,大
不,不
同,同
。,。


In [ ]:
# 潤句3：逐字偵測
# 適合用於產生銜接句+潤句
def convert2text(predictions,k=1):
    probs, indices = torch.topk(torch.softmax(predictions, -1), k)
    predicted_tokens = tokenizer.convert_ids_to_tokens(indices.tolist())
    return predicted_tokens,probs


from transformers import BertForMaskedLM

#
maskedLM_model = BertForMaskedLM.from_pretrained(model_version)
clear_output()

tokens, input_ids ,token_type_ids = get_input_from_mask_sentence(text_d)
for idx,tmp_token in enumerate(tokens):
    tmp_text = tokens[:]
    tmp_text[idx]="[MASK]"
    tmp_tokens, tmp_input_ids ,tmp_token_type_ids = get_input_from_mask_sentence("".join(tmp_text))
    with torch.no_grad():
        outputs = maskedLM_model(tmp_input_ids, token_type_ids=tmp_token_type_ids)
        predictions = outputs[0]
        # (1, seq_len, num_hidden_units)
    pred_token,prob = convert2text(predictions[0,idx])
    if prob[0] > magic_threshold and tokens[idx]!=pred_token[0]:
        print(f"{tokens[idx]},{pred_token[0]} --> {prob}")
        tokens[idx] = str(pred_token[0])
    elif tokens[idx]=="[MASK]":
        print(f"{tokens[idx]},{pred_token[0]} --> {prob}")
        tokens[idx] = str(pred_token[0])
    else:
        print(f"{tokens[idx]},{tokens[idx]}")
del maskedLM_model